This notebook is about determining a region around which we will taking account of the FISH, we don't want to detect them in regions that are very far away from the cell bodies.
 This region can be determined in several ways:

- If we do not have access to the cell body segmentation, we can proceed by applying a morphological dilation of a certain extent around the nuclei. If we do have access to the segmentations of the cell bodies we can proceed in the same way.
- We might want to do it by hand when there are several blobs and inhomogeneous regions in the same image.


23/04/25     Jacques Bourg @ Florian Muller lab. Institut Pasteur.

<div style="background-color: white; padding: 10px;">
    <img src="./pipeline.png" alt="pipeline" width="1200" height="420">
</div>

In [ ]:
import os
import sys
import numpy as np
from pathlib import Path
import skimage.io as io
import napari

import ipywidgets as widgets
from IPython.display import display

In [ ]:
%load_ext autoreload
%autoreload 2

base_dir = Path("../../src").resolve()
sys.path.append(str(base_dir))
sys.path.append(str(base_dir / "utils"))
sys.path.append(str(base_dir / "detection_fish"))
sys.path.append(str(base_dir / "segmentation"))

from utils.parameters_tracking import Parameter_tracking as Track
from utils.file_handling import FileProcessor
from segmentation.refine_seg import Segmentation
from segmentation.manual_roi_segm import ManualSegmentation as ManSeg

tk  = Track()
sg  = Segmentation()
fp  = FileProcessor()

In [ ]:
var = str(Path('../Analysis'))
batch_folders = os.listdir(var)
dropdown = widgets.Dropdown(options=batch_folders, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown)

In [ ]:
n         = np.where(np.array(batch_folders) == dropdown.value)[0][0]
file_path = str(Path(var) / Path(batch_folders[n]) / Path(batch_folders[n] +'.json'))
constants = tk.load_json(file_path)
batch_name= constants['BATCH_NAME']; print(batch_name)

In [ ]:
modalities = constants['MODALITIES']
dropdown2 = widgets.Dropdown(options=modalities, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown2)

In [ ]:
n2  = np.where(np.array(modalities) == dropdown2.value)[0][0]
modality = modalities[n2]
print(modality)

####  Cellbody masks:  whether the dapi with cell mask or the FISH were used to do the cellbody segmentations 

In [ ]:
channel_cell = constants['CHANNELS']
dropdown3 = widgets.Dropdown(options=channel_cell, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown3)

In [ ]:
n3     = np.where(np.array(channel_cell) == dropdown3.value)[0][0]
chan_c = channel_cell[n3]
print(chan_c)

Always choose CELL for the structures in the next cell !

In [ ]:
structs   = constants['STRUCTURES']
dropdown4 = widgets.Dropdown(options=structs, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown4)

In [ ]:
n4    = np.where(np.array(structs) == dropdown4.value)[0][0]
struc = structs[n4]; print(struc)

In [ ]:
batch_mip           = constants[f'BATCH_{modality}_{chan_c}_{struc}_MIP']
mask_cell_list_path = Path(f"../Analysis/{batch_name}/{modality}/segmentations_nuclei_cell_matched/{struc}/{batch_name}_masks_cells_matched.npy")
dict_mask_cells     = fp.load_masks_distributed_files(mask_cell_list_path)

In [ ]:
fish_area_folder    = Path(f"../Analysis/{batch_name}/{modality}/segmentations_nuclei_cell_matched/fish_area")    
if not fish_area_folder.exists():
    fish_area_folder.mkdir(parents=True)
mask_fish_area_path = Path(f"../Analysis/{batch_name}/{modality}/segmentations_nuclei_cell_matched/fish_area/{batch_name}_masks_fish_area.npy")

## A

#### AUTOMATIC DILATION (dilating cell masks)   _________________________________     to make a manual segmentation jump to the option B

In [ ]:
distance_dilation_fish_area = 1     # dilate the cells ROI by this quantity (in pixels)
 
dict_mask_fish  = {}
viewer6  = napari.Viewer(title="FISH AREA")
counter  = 0
for ind, file in enumerate(batch_mip):
    file            = Path(file)
    file_name       = str(file.stem)
    im              = io.imread(file)
    basename        = '_'.join(Path(file).stem.split('_')[:-3])  # the dict key of disk_mask_fish should be the basename (independent of which channel was used to compute the fish area) 
    masks_cells     = dict_mask_cells[basename]                                      
                                                      
    masks_b         = (masks_cells>0)*1  # bin masks
    masks_fish      = sg.dilate_labels(masks_b, distance=distance_dilation_fish_area)

    val             = np.percentile(im, 99)
    viewer6.add_image(im, rgb=False, name=f"MIP  {file_name}", contrast_limits=[0, val])
    viewer6.add_labels(masks_b, name=f"MASKS CELLS AREA {file_name}", opacity=.3, blending='translucent')
    fish_layer = viewer6.add_labels(masks_fish, name=f"FISH AREA {file_name}", opacity=.3, blending='translucent')
    if counter != 0:
        viewer6.layers[f"MIP  {file_name}"].visible             = False
        viewer6.layers[f"FISH AREA {file_name}"].visible        = False 
        viewer6.layers[f"MASKS CELLS AREA {file_name}"].visible = False 

    counter +=1
    dict_mask_fish[basename] = masks_fish
 
fp.save_masks_distributed_files(mask_fish_area_path, dict_mask_fish)

##### turn important lower case variables into uppercase for tracking, create those variables names with this content: useful for experiment tracking

In [ ]:
exec(f"MASK_{modality}_FISH_AREA_PATH = mask_fish_area_path", globals())
exec(f"DISTANCE_DILATION_{modality}_FISH_AREA = distance_dilation_fish_area", globals())

## B
MANUAL Segmentation of the FISH AREA of interests

In [ ]:
r         = io.imread(batch_mip[0])
ms        = ManSeg(batch_mip, np.shape(r)[-2:])
ms.run()

In [ ]:
dict_mask_fish = ms.dict_masks
np.save(mask_fish_area_path, dict_mask_fish)

In [ ]:
exec(f"MASK_{modality}_FISH_AREA_PATH = mask_fish_area_path", globals())

##   End of A or B

In [ ]:
constants2 = tk.collect_constants()
tk.save_constants_and_commit_hash(constants2, batch_name, folder_path = Path(f"../Analysis/{batch_name}"))